<a href="https://colab.research.google.com/github/suyashkrishangarg/Bard-api-for-everyone-/blob/main/quickstarts/rest/Prompting_REST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import secrets
import threading
import queue
import os
import time
import requests

# Precomputed hex digits set.
HEX_DIGITS = set("0123456789abcdef")

# Global shared state.
seen_tokens = set()                  # For uniqueness checks.
tokens_queue = queue.Queue()         # Queue for tokens waiting for validation.
valid_tokens_queue = queue.Queue()   # Queue for tokens that passed validation.
stop_event = threading.Event()       # Event to signal shutdown.

# Global counter for tokens generated.
tokens_count = 0
counter_lock = threading.Lock()

# Parameters.
WRITER_BATCH_SIZE = 1              # Number of tokens to write per I/O operation.
VALIDATOR_THREAD_COUNT = max((os.cpu_count() or 4) - 1, 1)  # Number of validator threads.
VALIDATION_URL = "https://openrouter.ai/api/v1/auth/key"       # URL for token validation.

def generate_hex_token():
    """Generates a 64-character hexadecimal string (32 bytes)."""
    return secrets.token_hex(32)

def contains_all_hex_chars(token):
    """Checks if the token contains every hexadecimal digit (0-9 and a-f)."""
    return HEX_DIGITS.issubset(token)

def token_generator():
    """Single generator thread: generates tokens and enqueues them for validation."""
    global tokens_count
    while not stop_event.is_set():
        token_body = generate_hex_token()
        if not contains_all_hex_chars(token_body):
            continue
        token = "sk-or-v1-" + token_body
        # Check uniqueness (no lock needed since this is the only generator thread).
        if token in seen_tokens:
            continue
        seen_tokens.add(token)
        tokens_queue.put(token)
        with counter_lock:
            tokens_count += 1

def is_token_valid(token):
    """
    Checks token validity using the external API.
    The token is sent as part of the Bearer authentication header.
    Returns True if the token is valid (HTTP 200); otherwise, prints the error and returns False.
    """
    headers = {
        "Authorization": f"Bearer {token}"
    }
    try:
        response = requests.get(VALIDATION_URL, headers=headers, timeout=5)
        if response.status_code == 200:
            return True
        else:
            print(f"Error: {response.status_code}, {response.text}")
            return False
    except Exception as e:
        print(f"Exception during token validation: {e}")
        return False

def validator_worker():
    """Worker thread for token validity checking."""
    while not stop_event.is_set() or not tokens_queue.empty():
        try:
            token = tokens_queue.get(timeout=1)
        except queue.Empty:
            continue
        if is_token_valid(token):
            valid_tokens_queue.put(token)
        tokens_queue.task_done()

def file_writer(file_path, batch_size=WRITER_BATCH_SIZE):
    """Dedicated writer thread: writes valid tokens to file in batches."""
    with open(file_path, "a") as file:
        buffer = []
        while not stop_event.is_set() or not valid_tokens_queue.empty():
            try:
                token = valid_tokens_queue.get(timeout=1)
                buffer.append(token)
                valid_tokens_queue.task_done()
                if len(buffer) >= batch_size:
                    file.write("\n".join(buffer) + "\n")
                    file.flush()
                    buffer.clear()
            except queue.Empty:
                if buffer:
                    file.write("\n".join(buffer) + "\n")
                    file.flush()
                    buffer.clear()

def monitor_speed():
    """Monitors and prints the token generation speed per second."""
    global tokens_count
    previous_count = 0
    while not stop_event.is_set():
        time.sleep(1)
        with counter_lock:
            current_count = tokens_count
        speed = current_count - previous_count
        previous_count = current_count
        print(f"Tokens generated per second: {speed}")

def main():
    file_path = "tokens.txt"
    threads = []

    # Start the single token generator thread.
    gen_thread = threading.Thread(target=token_generator)
    gen_thread.start()
    threads.append(gen_thread)

    # Start validator worker threads.
    validator_threads = []
    for _ in range(VALIDATOR_THREAD_COUNT):
        t = threading.Thread(target=validator_worker)
        t.start()
        validator_threads.append(t)
        threads.append(t)

    # Start the file writer thread.
    writer_thread = threading.Thread(target=file_writer, args=(file_path,))
    writer_thread.start()
    threads.append(writer_thread)

    # Start the monitor speed thread.
    monitor_thread = threading.Thread(target=monitor_speed)
    monitor_thread.start()
    threads.append(monitor_thread)

    try:
        while True:
            time.sleep(0.1)
    except KeyboardInterrupt:
        stop_event.set()
        # Wait for all threads to finish.
        gen_thread.join()
        for t in validator_threads:
            t.join()
        writer_thread.join()
        monitor_thread.join()
        print("Token generation and validation stopped by user.")

if __name__ == "__main__":
    main()


Error: 401, {"error":{"message":"Invalid credentials","code":401}}
Error: 401, {"error":{"message":"Invalid credentials","code":401}}
Error: 401, {"error":{"message":"Invalid credentials","code":401}}
Error: 401, {"error":{"message":"Invalid credentials","code":401}}
Tokens generated per second: 13307
Error: 401, {"error":{"message":"Invalid credentials","code":401}}
Error: 401, {"error":{"message":"Invalid credentials","code":401}}
Error: 401, {"error":{"message":"Invalid credentials","code":401}}
Error: 401, {"error":{"message":"Invalid credentials","code":401}}
Error: 401, {"error":{"message":"Invalid credentials","code":401}}
Error: 401, {"error":{"message":"Invalid credentials","code":401}}
Error: 401, {"error":{"message":"Invalid credentials","code":401}}
Error: 401, {"error":{"message":"Invalid credentials","code":401}}
Error: 401, {"error":{"message":"Invalid credentials","code":401}}Error: 401, {"error":{"message":"Invalid credentials","code":401}}

Error: 401, {"error":{"mess